In [1]:
import time
import random
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime

### CREACIÓN DEL NAVEGADOR REAL

In [2]:
# ============================================================================
# CONFIGURACIÓN INICIAL
# ============================================================================

# Número de páginas a recorrer
PAGINAS_A_RECORRER = 15

# ============================================================================
# FILTROS - PALABRAS CLAVE
# ============================================================================

# WHITELIST: Palabras que SÍ queremos (datos, análisis, tech)
palabras_clave_datos = (
    # Análisis (muy común en Colombia)
    r"analista.*dato|analista.*información|analista.*negocio|"
    r"analista.*business|business.*analyst|analista.*bi|"
    r"analista.*financiero|analista.*riesgo|analista.*crédito|"
    
    # Data en inglés (menos común pero existe)
    r"data\s+analyst|data\s+engineer|data\s+scientist|"
    
    # Tecnologías específicas
    r"\bsql\b|\bpython\b|power\s*bi|tableau|excel.*avanzado|"
    r"excel.*macros|vba\b|r\s+studio|rstudio|"
    
    # BI y reportería
    r"business\s+intelligence|inteligencia.*negocio|"
    r"reportería|reporting|dashboard|"
    
    # Big Data y ML (raro pero existe)
    r"big\s+data|machine\s+learning|científico.*dato|"
    
    # Bases de datos
    r"base.*dato.*sql|administrador.*base.*dato|"
    r"ingeniero.*dato|desarrollador.*bi|"
    
    # Términos amplios pero relacionados
    r"analítica|analytics|estadística.*aplicada|"
    r"modelamiento|modelación.*dato"
)

# BLACKLIST: Palabras que NO queremos
palabras_blacklist = (
    # 1. VENTAS Y COMERCIAL
    r"asesor.*comercial|ejecutivo.*comercial|vendedor|gestor.*comercial|"
    r"representante.*venta|líder.*venta|supervisor.*venta|"
    r"impulsa|promotor|televentas|preventa|posventa|"
    
    # 2. CALL CENTER Y ATENCIÓN AL CLIENTE
    r"call\s*center|contact\s*center|teleoperador|"
    r"agente.*servicio|asesor.*telefónico|"
    r"back\s*office.*admin|recepcionista|telemarketing|"
    
    # 3. BILINGÜE (generalmente call center)
    r"bilingüe|bilingual|english.*required|"
    r"nivel.*inglés.*c1|nivel.*inglés.*b2|"
    
    # 4. SALUD Y ENFERMERÍA
    r"enfermer|auxiliar.*salud|médico|odontólogo|"
    r"fisioterapeuta|terapeuta|cuidador|enfermero|"
    r"bacteriólogo|nutricionista|fonoaudiólogo|"
    
    # 5. SERVICIOS GENERALES Y OPERATIVOS
    r"mensajero|conductor|chofer|domiciliario|repartidor|"
    r"vigilante|guarda|seguridad|aseo|limpieza|"
    r"operario|auxiliar.*bodega|auxiliar.*almacén|"
    r"estibador|montacarguista|ayudante.*general|"
    
    # 6. GASTRONOMÍA Y HOTELERÍA
    r"chef|cocinero|mesero|bartender|barista|"
    r"auxiliar.*cocina|steward|cajero.*restaurante|"
    r"parrillero|pizzero|ayudante.*cocina|"
    
    # 7. CONTABILIDAD OPERATIVA (no analítica)
    r"auxiliar.*contable|asistente.*contable|"
    r"contador.*junior(?!\s+data)|facturador|"
    r"tesorero|cartera|cajero(?!\s+data)|"
    
    # 8. ADMINISTRATIVO BÁSICO (no estratégico)
    r"recepcionista|secretaria|auxiliar.*admin.*(?!.*data|.*bi)|"
    r"archivador|mensajería.*interna|"
    r"asistente.*dirección(?!.*data)|"
    
    # 9. RECURSOS HUMANOS OPERATIVO
    r"auxiliar.*nómina|auxiliar.*talento|"
    r"reclutador.*masivo|gestor.*personal.*temporal|"
    
    # 10. PRODUCCIÓN Y MANUFACTURA
    r"operador.*producción|supervisor.*planta|"
    r"operador.*máquina|técnico.*mantenimiento|"
    r"soldador|mecánico|electricista|tornero|"
    
    # 11. CONSTRUCCIÓN
    r"maestro.*obra|oficial.*construcción|albañil|"
    r"plomero|electricista.*residencial|ayudante.*construcción|"
    
    # 12. LEGAL BÁSICO
    r"auxiliar.*jurídico|asistente.*legal(?!.*compliance.*data)|"
    r"archivador.*jurídico|"
    
    # 13. LOGÍSTICA OPERATIVA
    r"auxiliar.*logística(?!.*data|.*bi)|"
    r"coordinador.*despachos(?!.*data)|"
    r"auxiliar.*inventario(?!.*data)|"
    
    # 14. EDUCACIÓN BÁSICA
    r"docente|profesor|maestro.*primaria|"
    r"auxiliar.*pedagógico|niñera|"
    
    # 15. OTROS SERVICIOS
    r"estilista|peluquero|manicurista|masajista|"
    r"fotógrafo.*eventos|diseñador.*gráfico.*junior(?!.*data)"
)

# Compilar patrones (más eficiente)
patron_si = re.compile(rf"(?i)({palabras_clave_datos})")
patron_no = re.compile(rf"(?i)\b({palabras_blacklist})\b")

In [3]:
# ============================================================================
# INICIAR SELENIUM
# ============================================================================

print("🚀 Iniciando navegador Chrome...")
opciones_chrome = Options()
opciones_chrome.add_argument("--start-maximized")
opciones_chrome.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
opciones_chrome.add_argument("--disable-blink-features=AutomationControlled")
opciones_chrome.add_experimental_option("excludeSwitches", ["enable-automation"])
opciones_chrome.add_experimental_option('useAutomationExtension', False)

servicio = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servicio, options=opciones_chrome)

print("✅ Navegador iniciado correctamente\n")

# ============================================================================
# VARIABLES PARA ALMACENAR RESULTADOS
# ============================================================================

ofertas_aceptadas = []          # Lista principal de ofertas que PASAN los filtros
ofertas_rechazadas_blacklist = []   # Ofertas rechazadas por estar en blacklist
ofertas_rechazadas_sin_keywords = [] # Ofertas rechazadas por no tener keywords
ofertas_todas = []              # TODAS las ofertas sin filtrar (para análisis)
enlaces_procesados = set()      # Para evitar duplicados
estadisticas_por_pagina = []    # Métricas de cada página

url_base = "https://co.computrabajo.com/empleos-en-bogota-dc?pubdate=1"

# ============================================================================
# PROCESO DE SCRAPING
# ============================================================================

print("="*70)
print("🔍 INICIANDO BÚSQUEDA DE OFERTAS DE DATOS EN BOGOTÁ")
print("="*70)

try:
    for numero_pagina in range(1, PAGINAS_A_RECORRER + 1):
        # Construir URL
        if numero_pagina > 1:
            url = f"{url_base}&p={numero_pagina}"
        else:
            url = url_base
        
        print(f"\n📄 Página {numero_pagina}/{PAGINAS_A_RECORRER}...")
        
        # Navegar a la página
        navegador.get(url)
        tiempo_espera = random.uniform(4, 7)
        time.sleep(tiempo_espera)
        
        # Parsear HTML
        soup = BeautifulSoup(navegador.page_source, 'html.parser')
        tarjetas_ofertas = soup.find_all('article', class_='box_offer')
        
        # Contadores para esta página
        aceptadas_pagina = 0
        rechazadas_blacklist_pagina = 0
        rechazadas_sin_keywords_pagina = 0
        
        print(f"   🔎 Encontradas {len(tarjetas_ofertas)} ofertas en la página")
        
        # Procesar cada oferta
        for tarjeta in tarjetas_ofertas:
            try:
                # Extraer título
                tag_titulo = tarjeta.find('a', class_='js-o-link')
                if not tag_titulo:
                    continue
                
                titulo = tag_titulo.get_text(strip=True)
                href = tag_titulo.get('href', '')
                
                if not href:
                    continue
                
                link = "https://co.computrabajo.com" + href
                
                # Evitar duplicados
                if link in enlaces_procesados:
                    continue
                
                enlaces_procesados.add(link)
                
                # Extraer empresa
                tag_empresa = tarjeta.find('p', class_='fs16')
                if not tag_empresa:
                    tag_empresa = tarjeta.find('span', class_='fc_base')
                empresa = tag_empresa.get_text(strip=True) if tag_empresa else "Confidencial"
                
                # Extraer fecha
                tag_fecha = tarjeta.find('span', class_='fc_verde')
                if not tag_fecha:
                    tag_fecha = tarjeta.find('p', class_='fs13 fc_base mt5')
                fecha = tag_fecha.get_text(strip=True) if tag_fecha else "Hoy"
                
                # Extraer ubicación
                tag_ubicacion = tarjeta.find('p', class_='fs13 fc_base')
                ubicacion = tag_ubicacion.get_text(strip=True) if tag_ubicacion else "Bogotá"
                
                # Crear diccionario con toda la info
                info_oferta = {
                    'titulo': titulo,
                    'empresa': empresa,
                    'ubicacion': ubicacion,
                    'fecha': fecha,
                    'link': link,
                    'pagina': numero_pagina,
                    'timestamp_scraping': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }
                
                # Guardar en TODAS (sin filtrar)
                ofertas_todas.append(info_oferta.copy())
                
                # ============================================================
                # APLICAR FILTROS
                # ============================================================
                
                # PASO 1: ¿Está en la blacklist?
                resultado_blacklist = patron_no.search(titulo)
                
                if resultado_blacklist:
                    # RECHAZADA por blacklist
                    palabra_problema = resultado_blacklist.group()
                    info_oferta['razon_rechazo'] = f'Blacklist: {palabra_problema}'
                    ofertas_rechazadas_blacklist.append(info_oferta)
                    rechazadas_blacklist_pagina += 1
                    print(f"      🚫 {titulo[:45]}... [{palabra_problema}]")
                    continue
                
                # PASO 2: ¿Tiene keywords de datos?
                resultado_whitelist = patron_si.search(titulo)
                
                if resultado_whitelist:
                    # ✅ ACEPTADA
                    keyword_encontrada = resultado_whitelist.group()
                    info_oferta['keyword_match'] = keyword_encontrada
                    ofertas_aceptadas.append(info_oferta)
                    aceptadas_pagina += 1
                    print(f"      ✅ {titulo}")
                else:
                    # RECHAZADA por no tener keywords
                    info_oferta['razon_rechazo'] = 'Sin keywords de datos'
                    ofertas_rechazadas_sin_keywords.append(info_oferta)
                    rechazadas_sin_keywords_pagina += 1
                    print(f"      ⏭️ {titulo[:45]}...")
            
            except Exception as error:
                print(f"      ❌ Error procesando tarjeta: {str(error)[:50]}")
                continue
        
        # Guardar estadísticas de la página
        stats_pagina = {
            'numero_pagina': numero_pagina,
            'ofertas_encontradas': len(tarjetas_ofertas),
            'aceptadas': aceptadas_pagina,
            'rechazadas_blacklist': rechazadas_blacklist_pagina,
            'rechazadas_sin_keywords': rechazadas_sin_keywords_pagina,
            'tiempo_espera': round(tiempo_espera, 2)
        }
        estadisticas_por_pagina.append(stats_pagina)
        
        # Resumen de la página
        print(f"\n   📊 Resumen página {numero_pagina}:")
        print(f"      ✅ Aceptadas: {aceptadas_pagina}")
        print(f"      🚫 Rechazadas (blacklist): {rechazadas_blacklist_pagina}")
        print(f"      ⏭️ Rechazadas (sin keywords): {rechazadas_sin_keywords_pagina}")
        print(f"      📈 Total acumulado aceptadas: {len(ofertas_aceptadas)}")
        
        # Guardado intermedio cada 5 páginas
        if numero_pagina % 5 == 0 and ofertas_aceptadas:
            df_backup = pd.DataFrame(ofertas_aceptadas)
            archivo_backup = f'backup_pagina_{numero_pagina}.csv'
            print(f"      💾 Backup guardado: {archivo_backup}")

except KeyboardInterrupt:
    print("\n\n⚠️ Proceso interrumpido manualmente por el usuario")

except Exception as error_general:
    print(f"\n\n❌ Error general: {str(error_general)}")
    import traceback
    traceback.print_exc()

finally:
    print("\n🔒 Cerrando navegador...")
    navegador.quit()

🚀 Iniciando navegador Chrome...
✅ Navegador iniciado correctamente

🔍 INICIANDO BÚSQUEDA DE OFERTAS DE DATOS EN BOGOTÁ

📄 Página 1/15...
   🔎 Encontradas 20 ofertas en la página
      ⏭️ Auxiliar Motorizado para trabajar en Telecomu...
      🚫 Coordinador de Seguridad Parqueadero... [Seguridad]
      🚫 Asesor en ventas call center idioma portugués... [call center]
      🚫 Auxiliares de Bodega, ayudante logístico, ali... [Auxiliares de Bodega]
      🚫 Conductor repartidor... [Conductor]
      ⏭️ Auxiliar de farmacia Nocturno Bogotá Zona Nor...
      ⏭️ Carpintero e instalador avanzado Bogotá...
      ⏭️ ¡Buscamos Analista de nomina!...
      ⏭️ Tecnico de mantenimiento industrial...
      🚫 Operario logístico Materias Primas... [Operario]
      🚫 Operario de Producción –Experiencia sustancia... [Operario]
      ⏭️ Cajeros, tiempo completo disponibilidad inmed...
      ⏭️ Facilitador de Servicio al cliente...
      🚫 Tecnico Electricista Automotriz... [Electricista]
      ⏭️ Licenciado/a

In [5]:
# ============================================================================
# GUARDAR TODOS LOS RESULTADOS
# ============================================================================

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

print("\n" + "="*70)
print("💾 GUARDANDO RESULTADOS...")
print("="*70)

# 1. OFERTAS ACEPTADAS (las buenas)
df_aceptadas = pd.DataFrame(ofertas_aceptadas)
# Eliminar duplicados por si acaso
df_aceptadas = df_aceptadas.drop_duplicates(subset=['link'], keep='first')
archivo_aceptadas = f'ofertas_ACEPTADAS_{timestamp}.csv'
print(f"✅ Ofertas ACEPTADAS: {len(df_aceptadas)} → {archivo_aceptadas}")

# 2. OFERTAS RECHAZADAS POR BLACKLIST
df_rechazadas_bl = pd.DataFrame(ofertas_rechazadas_blacklist)
archivo_rechazadas_bl = f'ofertas_RECHAZADAS_blacklist_{timestamp}.csv'
print(f"🚫 Ofertas rechazadas (blacklist): {len(df_rechazadas_bl)} → {archivo_rechazadas_bl}")

# 3. OFERTAS RECHAZADAS POR NO TENER KEYWORDS
df_rechazadas_kw = pd.DataFrame(ofertas_rechazadas_sin_keywords)
archivo_rechazadas_kw = f'ofertas_RECHAZADAS_sin_keywords_{timestamp}.csv'
print(f"⏭️ Ofertas rechazadas (sin keywords): {len(df_rechazadas_kw)} → {archivo_rechazadas_kw}")


# 4. TODAS LAS OFERTAS (sin filtrar)
df_todas = pd.DataFrame(ofertas_todas)
df_todas = df_todas.drop_duplicates(subset=['link'], keep='first')

# 5. ESTADÍSTICAS POR PÁGINA
df_stats = pd.DataFrame(estadisticas_por_pagina)
archivo_stats = f'estadisticas_por_pagina_{timestamp}.csv'
print(f"📊 Estadísticas por página: {archivo_stats}")

# ============================================================================
# RESUMEN FINAL
# ============================================================================

print("\n" + "="*70)
print("📊 RESUMEN FINAL DEL SCRAPING")
print("="*70)

total_ofertas_procesadas = len(ofertas_todas)
total_aceptadas = len(ofertas_aceptadas)
total_rechazadas_bl = len(ofertas_rechazadas_blacklist)
total_rechazadas_kw = len(ofertas_rechazadas_sin_keywords)
total_rechazadas = total_rechazadas_bl + total_rechazadas_kw

print(f"\n📈 Ofertas procesadas: {total_ofertas_procesadas}")
print(f"   ✅ Aceptadas: {total_aceptadas} ({total_aceptadas/total_ofertas_procesadas*100:.1f}%)" if total_ofertas_procesadas > 0 else "   ✅ Aceptadas: 0")
print(f"   🚫 Rechazadas: {total_rechazadas} ({total_rechazadas/total_ofertas_procesadas*100:.1f}%)" if total_ofertas_procesadas > 0 else "   🚫 Rechazadas: 0")
print(f"      • Por blacklist: {total_rechazadas_bl}")
print(f"      • Sin keywords: {total_rechazadas_kw}")

# Top palabras de blacklist que más rechazaron
if total_rechazadas_bl > 0:
    print("\n🚫 TOP 10 PALABRAS DE BLACKLIST QUE MÁS RECHAZARON:")
    print("-" * 70)
    razones = [oferta['razon_rechazo'] for oferta in ofertas_rechazadas_blacklist]
    conteo_razones = {}
    for razon in razones:
        conteo_razones[razon] = conteo_razones.get(razon, 0) + 1
    
    top_razones = sorted(conteo_razones.items(), key=lambda x: x[1], reverse=True)[:10]
    for razon, cantidad in top_razones:
        print(f"   • {razon}: {cantidad} ofertas")

print("\n" + "="*70)
print("✨ PROCESO COMPLETADO")
print("="*70)

# ============================================================================
# VARIABLES DISPONIBLES PARA INSPECCIÓN
# ============================================================================

print("\n💡 VARIABLES DISPONIBLES PARA INSPECCIÓN:")
print("   • ofertas_aceptadas           → Lista con ofertas que pasaron filtros")
print("   • ofertas_rechazadas_blacklist → Lista con ofertas rechazadas por blacklist")
print("   • ofertas_rechazadas_sin_keywords → Lista con ofertas sin keywords")
print("   • df_aceptadas                → DataFrame de ofertas aceptadas")
print("   • df_rechazadas_kw            → DataFrame de rechazadas sin keywords")
print("   • df_todas                    → DataFrame de todas las ofertas")
print("   • enlaces_procesados          → Set con todos los links procesados")
print("   • patron_si                   → Patrón regex de whitelist")
print("   • patron_no                   → Patrón regex de blacklist")


💾 GUARDANDO RESULTADOS...
✅ Ofertas ACEPTADAS: 3 → ofertas_ACEPTADAS_20260214_094631.csv
🚫 Ofertas rechazadas (blacklist): 130 → ofertas_RECHAZADAS_blacklist_20260214_094631.csv
⏭️ Ofertas rechazadas (sin keywords): 167 → ofertas_RECHAZADAS_sin_keywords_20260214_094631.csv
📊 Estadísticas por página: estadisticas_por_pagina_20260214_094631.csv

📊 RESUMEN FINAL DEL SCRAPING

📈 Ofertas procesadas: 300
   ✅ Aceptadas: 3 (1.0%)
   🚫 Rechazadas: 297 (99.0%)
      • Por blacklist: 130
      • Sin keywords: 167

🚫 TOP 10 PALABRAS DE BLACKLIST QUE MÁS RECHAZARON:
----------------------------------------------------------------------
   • Blacklist: Call Center: 15 ofertas
   • Blacklist: Asesor comercial: 13 ofertas
   • Blacklist: call center: 12 ofertas
   • Blacklist: Operario: 8 ofertas
   • Blacklist: Bilingüe: 8 ofertas
   • Blacklist: Asesor Comercial: 6 ofertas
   • Blacklist: Conductor: 5 ofertas
   • Blacklist: Auxiliar Administrativo / Lunes a Viernes: 3 ofertas
   • Blacklist: Sold

In [6]:
df_aceptadas

,titulo,empresa,ubicacion,fecha,link,pagina,timestamp_scraping,keyword_match
0,Analista de datos/experiencia en bases de recu...,"4,4ATENTO S.A.",Bogotá,Hoy,https://co.computrabajo.com/ofertas-de-trabajo...,7,2026-02-14 09:44:13,Analista de dato
1,Reporting con experiencia en area contable o f...,"4,1CUSTOMER OPERATION SUCCES S.A.S.",Bogotá,Hoy,https://co.computrabajo.com/ofertas-de-trabajo...,7,2026-02-14 09:44:13,Reporting
2,Analista legal de Privacidad y Protección de ...,Importante empresa del sector Outsourcing,Bogotá,Hoy,https://co.computrabajo.com/ofertas-de-trabajo...,8,2026-02-14 09:44:20,Analista legal de Privacidad y Protección de ...


In [8]:
df_rechazadas_kw.to_excel(r"C:\Users\9 ----- SIG\Downloads\ofertas.xlsx", index = False) 